In [10]:
# Import the NumPy library for efficient numerical operations on arrays and integers.
import numpy as np

# Problem 2: Fractional Parts of Cube Roots




## Overview

- Purpose: compute fractional parts of cube roots of primes and derive SHA-256 K constants.
- Key functions: `primes(n)`, `cube_root_fractions`, `fractional32_hex`, `cube_root_constants`, `test_cube_root_constants`.
- Notes: Uses NumPy; compare results to FIPS reference values to verify correctness.

## `primes(n)`

- Purpose: Generate the first `n` prime numbers and return them as a Python list.
- Args: `n` (int) — number of primes to generate. If `n <= 0` an empty list is returned.
- Returns: `list[int]` containing the first `n` primes (e.g. `primes(4)` -> `[2, 3, 5, 7]`).
- Notes: This implementation uses simple trial division which is efficient enough for small `n` such as 64. For very large `n` a segmented sieve is preferred.
- Example: `primes(64)` produces the first 64 prime numbers required for the cube root constants.


In [11]:
def primes(n: int) -> list:
    """
    Return the first `n` prime numbers.

    This implementation uses simple trial division which is fine for small n
    (here n=64). It returns an empty list for n <= 0.
    """
    if not isinstance(n, int):
        raise TypeError("n must be an integer")
    if n <= 0:
        return []

    primes_list = [2]
    candidate = 3
    while len(primes_list) < n:
        is_prime = True
        for p in primes_list:
            if p * p > candidate:
                break
            if candidate % p == 0:
                is_prime = False
                break
        if is_prime:
            primes_list.append(candidate)
        candidate += 2
    return primes_list

# Example usage (commented out):
# print(primes(10))


In [12]:
print(primes(64)) # Example usage: print all primes less than 64.
print(primes(10)) # Example usage: print all primes less than 10.
print(primes(0))  # Example usage: print primes when n is 0 (should be empty list).


[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311]
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
[]


## Helper Functions
- `cube_root_fractions(prime_list)`: Returns fractional parts of the cube roots for the given primes as a NumPy array.
- `fractional32_hex(frac_array)`: Converts fractional parts to first 32 bits and formats each as an 8-character hex string.

In [13]:
# Compute the fractional parts of the cube roots for the given prime list.
def cube_root_fractions(prime_list: list) -> np.ndarray:
    """
    Compute the fractional parts of the cube roots for the given prime list.
    Returns a NumPy array of fractional parts (float64).
    """
    arr = np.array(prime_list, dtype=np.float64)
    c = np.cbrt(arr)
    frac = c - np.floor(c)

    return frac


In [14]:
# Extract the first 32 bits of each fractional part and format as 8-digit hex strings.
def fractional32_hex(frac_array: np.ndarray) -> list[str]:
    """
    Extract the first 32 bits of each fractional part and format as 8-digit hex strings.
    """
    words = (np.floor(frac_array * (2**32)).astype(np.uint64) & 0xFFFFFFFF)
    
    return [f"{int(w):08x}" for w in words]

## `cube_root_constants()`
- Purpose: Compute SHA-like cube-root `K` constants from the first 64 primes.
- Output: 64 hex strings (8 chars) — first 32 bits of each fractional cube root.
- Implementation: Uses helpers `cube_root_fractions(prime_list)` and `fractional32_hex(frac_array)`; behavior unchanged from original.
- Example: `cube_root_constants()` → `['428a2f98', '71374491', ...]`.

In [15]:
# Compute cube root K constants: first 32 bits of fractional part of cube roots
def cube_root_constants():
    """
    Compute cube root K constants: first 32 bits of fractional part of cube roots
    of the first 64 primes. Return list of hex strings.
    """
    # Get first 64 primes
    prime_list = primes(64)

    # Fractional parts of cube roots
    frac = cube_root_fractions(prime_list)

    # First 32 bits of fractional part formatted as hex
    computed_hex = fractional32_hex(frac)

    return computed_hex

In [16]:
print(cube_root_constants())  # Example usage: print cube root K constants.

['428a2f98', '71374491', 'b5c0fbcf', 'e9b5dba5', '3956c25b', '59f111f1', '923f82a4', 'ab1c5ed5', 'd807aa98', '12835b01', '243185be', '550c7dc3', '72be5d74', '80deb1fe', '9bdc06a7', 'c19bf174', 'e49b69c1', 'efbe4786', '0fc19dc6', '240ca1cc', '2de92c6f', '4a7484aa', '5cb0a9dc', '76f988da', '983e5152', 'a831c66d', 'b00327c8', 'bf597fc7', 'c6e00bf3', 'd5a79147', '06ca6351', '14292967', '27b70a85', '2e1b2138', '4d2c6dfc', '53380d13', '650a7354', '766a0abb', '81c2c92e', '92722c85', 'a2bfe8a1', 'a81a664b', 'c24b8b70', 'c76c51a3', 'd192e819', 'd6990624', 'f40e3585', '106aa070', '19a4c116', '1e376c08', '2748774c', '34b0bcb5', '391c0cb3', '4ed8aa4a', '5b9cca4f', '682e6ff3', '748f82ee', '78a5636f', '84c87814', '8cc70208', '90befffa', 'a4506ceb', 'bef9a3f7', 'c67178f2']


## `test_cube_root_constants()`
- Purpose: Verify computed constants against FIPS 180-4 reference values.
- Returns: Dict with `computed_hex`, `reference_hex`, and `mismatches` (list of `(index, computed, reference)`).
- Usage: `res = test_cube_root_constants(); print(res['mismatches'])` — empty list means all match.

In [17]:
def test_cube_root_constants():
    """
    Test the computed cube root K constants against FIPS 180-4 reference values.
    
    Returns a dict with computed_hex, reference_hex, and any mismatches found.
    """
    # Get computed constants
    computed_hex = cube_root_constants()
    
    # Reference K constants from FIPS 180-4 (first 64 words)
    reference_hex = [
        "428a2f98","71374491","b5c0fbcf","e9b5dba5","3956c25b","59f111f1","923f82a4","ab1c5ed5",
        "d807aa98","12835b01","243185be","550c7dc3","72be5d74","80deb1fe","9bdc06a7","c19bf174",
        "e49b69c1","efbe4786","0fc19dc6","240ca1cc","2de92c6f","4a7484aa","5cb0a9dc","76f988da",
        "983e5152","a831c66d","b00327c8","bf597fc7","c6e00bf3","d5a79147","06ca6351","14292967",
        "27b70a85","2e1b2138","4d2c6dfc","53380d13","650a7354","766a0abb","81c2c92e","92722c85",
        "a2bfe8a1","a81a664b","c24b8b70","c76c51a3","d192e819","d6990624","f40e3585","106aa070",
        "19a4c116","1e376c08","2748774c","34b0bcb5","391c0cb3","4ed8aa4a","5b9cca4f","682e6ff3",
        "748f82ee","78a5636f","84c87814","8cc70208","90befffa","a4506ceb","bef9a3f7","c67178f2",
    ]

    # Compare
    mismatches = []
    for i, (calc, ref) in enumerate(zip(computed_hex, reference_hex), start=1):
        if calc != ref:
            mismatches.append((i, calc, ref))

    # Return results
    return {
        "computed_hex": computed_hex,
        "reference_hex": reference_hex,
        "mismatches": mismatches,
    }

# Example usage: test cube_root_constants.
print(test_cube_root_constants()) 

{'computed_hex': ['428a2f98', '71374491', 'b5c0fbcf', 'e9b5dba5', '3956c25b', '59f111f1', '923f82a4', 'ab1c5ed5', 'd807aa98', '12835b01', '243185be', '550c7dc3', '72be5d74', '80deb1fe', '9bdc06a7', 'c19bf174', 'e49b69c1', 'efbe4786', '0fc19dc6', '240ca1cc', '2de92c6f', '4a7484aa', '5cb0a9dc', '76f988da', '983e5152', 'a831c66d', 'b00327c8', 'bf597fc7', 'c6e00bf3', 'd5a79147', '06ca6351', '14292967', '27b70a85', '2e1b2138', '4d2c6dfc', '53380d13', '650a7354', '766a0abb', '81c2c92e', '92722c85', 'a2bfe8a1', 'a81a664b', 'c24b8b70', 'c76c51a3', 'd192e819', 'd6990624', 'f40e3585', '106aa070', '19a4c116', '1e376c08', '2748774c', '34b0bcb5', '391c0cb3', '4ed8aa4a', '5b9cca4f', '682e6ff3', '748f82ee', '78a5636f', '84c87814', '8cc70208', '90befffa', 'a4506ceb', 'bef9a3f7', 'c67178f2'], 'reference_hex': ['428a2f98', '71374491', 'b5c0fbcf', 'e9b5dba5', '3956c25b', '59f111f1', '923f82a4', 'ab1c5ed5', 'd807aa98', '12835b01', '243185be', '550c7dc3', '72be5d74', '80deb1fe', '9bdc06a7', 'c19bf174', 'e4